In [ ]:
from tqdm import tqdm
import random
import numpy as np
from collections import namedtuple
from matplotlib import pyplot as plt
from itertools import combinations

from ensembler.geometry import measures
%matplotlib inline

import matplotlib
plot_layout_settings = {'font.family': 'sans-serif',
                        "font.serif": 'Times',
                        "font.size": 10,
                        'xtick.labelsize': 10,
                        'ytick.labelsize': 10,
                        'axes.labelsize': 12,
                        'axes.titlesize': 14,
                        'legend.fontsize': 8,
                        'savefig.dpi': 300,
                        'figure.facecolor' : "white",
                        'animation.html': 'jshtml',
                        }


for key, value in plot_layout_settings.items():
    matplotlib.rcParams[key] = value


In [ ]:
def draw_particles_2D(particles):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    all_x = [p.coordinates[0] for p in particles]
    all_y = [p.coordinates[1] for p in particles]

    ax.scatter(x=all_x, y=all_y, c="r", alpha=0.5)
    ax.set_ylabel("y")
    ax.set_xlabel("x")

    return fig

In [ ]:
lennard_jones = lambda epsilon,sigma, x_shift, y_shift: lambda position: 4 * epsilon * ((sigma / (position - x_shift)) ** 12 - (sigma / (position - x_shift)) ** 6) + y_shift



## Define Coordinates

In [ ]:
particle = namedtuple("particle", ["ID", "mass", "partialCharge", "coordinates"])
n_particles = 150
particles = []
min_x = min_y = 0
max_x = max_y = 10
for particleID in range(n_particles):
    x = random.randint(min_x*100, max_x*100)/100
    y = random.randint(min_y*100, max_y*100)/100
    coordI = np.array([x,y])
    particleI = particle(ID=particleID, mass=1, partialCharge=0, coordinates=coordI)

    particles.append(particleI)

draw_particles_2D(particles)
pass

## Define potential

In [ ]:
lj = lennard_jones(sigma=3, epsilon=5, x_shift=-2.0, y_shift=0)

positions = np.linspace(0,10, 100)
penes = list(map(lj, positions))
plt.plot(positions, penes)
plt.ylim([-20,20])

## build simulation

In [ ]:
def pbc(pos, space_range = [0,10]):
    if(pos>max(space_range)):
        return min(space_range)+(pos%max(space_range))
    elif(pos<min(space_range)):
        return max(space_range)-(min(space_range)%pos)
    else:
        return pos

vpbc = np.vectorize(pbc)
r0 = r1 =1
dt = 1
steps = 200
brownsch_shift = lambda : np.array([random.randint(0,100)/100, random.randint(0,100)/100])


#draw_particles_2D(particles)

sorted_particles = list(sorted(particles, key=lambda x: x.ID))
nParticles = len(particles)
nDim = 2

traj = []
tmp_particles_coords = np.array([p.coordinates for p in sorted_particles])
traj.append(tmp_particles_coords)

for step in tqdm(range(steps)):
        
    #drift:
    tmp_drift_particles_coords  = np.zeros([nParticles, nDim])
    for ind, p in enumerate(sorted_particles):     
        cparticle = tmp_particles_coords[ind]
        
        p_vel = [0,0] #brownsch_shift() #np.array([0, 0.2])#
        mean_coordsI = cparticle + p_vel
        pbc_coordsI = vpbc(mean_coordsI)
        tmp_drift_particles_coords[ind] = pbc_coordsI
    
    #N-Body-Potentials: 
    ##calculate new virtual position, energies
    new_particles_shifts = {}
    for pA,pB in combinations(sorted_particles,2):
        pointA, pointB = np.array(tmp_particles_coords[pA.ID]), np.array(tmp_particles_coords[pB.ID])
        
        #PBC In distances 
        r = pointB-pointA
        ene = lj(np.abs(r))
        shift_norm = (np.abs(np.array([r0, r1]))/measures.calculate_vector_length(r)*dt)/2

        if(not pA.ID in new_particles_shifts):
            new_particles_shifts.update({pA.ID : {"shift":[shift_norm], "ene":[ene]}})
        else:
            new_particles_shifts[pA.ID]["shift"].append(shift_norm)
            new_particles_shifts[pA.ID]["ene"].append(ene)

        if(not pB.ID in new_particles_shifts):
            new_particles_shifts.update({pB.ID : {"shift":[-shift_norm], "ene":[ene]}})
        else:
            new_particles_shifts[pB.ID]["shift"].append(-shift_norm)
            new_particles_shifts[pB.ID]["ene"].append(ene)

    ##update particles to new position
    new_particles_coords = np.zeros([nParticles, nDim])
    for ind, p in enumerate(sorted_particles):     
        cparticle = tmp_drift_particles_coords[ind]
        shiftsI = np.array(new_particles_shifts[p.ID]["shift"])
        enesI = np.array(new_particles_shifts[p.ID]["ene"])
        
        totale_shift = [x if(x<1) else 1 for x in  np.average(shiftsI.T, weights=enesI.T, axis=1)]
        
        mean_coordsI = cparticle + totale_shift
        pbc_coordsI = vpbc(mean_coordsI)
        new_particles_coords[ind] = pbc_coordsI
    tmp_particles_coords = np.array(new_particles_coords)

    traj.append(tmp_particles_coords)
traj = np.array(traj)
#draw_particles_2D(tmp_particles_coords)
pass

### Visualization

In [ ]:
from matplotlib import animation

def animation_particle_2D(particle_traj, out_path=None, 
                          x_range=None, y_range=None,
                          out_writer="pillow", dpi=120):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_ylabel("y")
    ax.set_xlabel("x")
    if(not x_range is None):
        ax.set_xlim([min(x_range), max(x_range)])
    if(not y_range is None):
        ax.set_ylim([min(y_range), max(y_range)])

    ax.plot(particle_traj[0, :, 0], particle_traj[0, :, 1], "bo", ms=5, c="b", lw=0, alpha=0.3)

    t0 = 0
    tmax = len(particle_traj)
    step_size = 1

    particles_points, = ax.plot([],[], "bo", ms=1.5, c="r", lw=0, alpha=0.4)
    all_data = particle_traj

    def init():
        particles_points.set_data(particle_traj[0, :, 0], particle_traj[0, :, 1])


    def data_gen(t=t0):
        while t < tmax:
            t+=step_size
            data = all_data[t-1]
            yield data

    def run(data):
        particles_points.set_data(data[:,0],data[:,1])
        return particles_points,

    ani = animation.FuncAnimation(fig=fig, func=run, frames=data_gen, init_func=init)
    return ani

In [ ]:
c_range = [-1,11]
ani = animation_particle_2D(particle_traj=traj, x_range=c_range, y_range=c_range)
ani

In [ ]:
ani.save("fun.mpg")

## Class

In [ ]:
class particle_system():

    def __init__(self, particle_coord, particle_ff, integrator):
        self.particle = particle_coord
        self.ff = particle_ff
        self.integrator = integrator


            
    def simulate(self, steps = 10):
        for step in tqdm(range(steps)):

            #drift:
            tmp_drift_particles_coords  = np.zeros([nParticles, nDim])
            for ind, p in enumerate(sorted_particles):     
                cparticle = tmp_particles_coords[ind]

                p_vel = [0,0] #brownsch_shift() #np.array([0, 0.2])#
                mean_coordsI = cparticle + p_vel
                pbc_coordsI = vpbc(mean_coordsI)
                tmp_drift_particles_coords[ind] = pbc_coordsI

            #N-Body-Potentials: 
            ##calculate new virtual position, energies
            new_particles_shifts = {}
            for pA,pB in combinations(sorted_particles,2):
                pointA, pointB = np.array(tmp_particles_coords[pA.ID]), np.array(tmp_particles_coords[pB.ID])

                #PBC In distances 
                r = pointB-pointA
                ene = lj(np.abs(r))
                shift_norm = (np.abs(np.array([r0, r1]))/measures.calculate_vector_length(r)*dt)/2

                if(not pA.ID in new_particles_shifts):
                    new_particles_shifts.update({pA.ID : {"shift":[shift_norm], "ene":[ene]}})
                else:
                    new_particles_shifts[pA.ID]["shift"].append(shift_norm)
                    new_particles_shifts[pA.ID]["ene"].append(ene)

                if(not pB.ID in new_particles_shifts):
                    new_particles_shifts.update({pB.ID : {"shift":[-shift_norm], "ene":[ene]}})
                else:
                    new_particles_shifts[pB.ID]["shift"].append(-shift_norm)
                    new_particles_shifts[pB.ID]["ene"].append(ene)

            ##update particles to new position
            new_particles_coords = np.zeros([nParticles, nDim])
            for ind, p in enumerate(sorted_particles):     
                cparticle = tmp_drift_particles_coords[ind]
                shiftsI = np.array(new_particles_shifts[p.ID]["shift"])
                enesI = np.array(new_particles_shifts[p.ID]["ene"])

                totale_shift = [x if(x<1) else 1 for x in  np.average(shiftsI.T, weights=enesI.T, axis=1)]

                mean_coordsI = cparticle + totale_shift
                pbc_coordsI = vpbc(mean_coordsI)
                new_particles_coords[ind] = pbc_coordsI
            tmp_particles_coords = np.array(new_particles_coords)

            traj.append(tmp_particles_coords)
        traj = np.array(traj)

